#### Modelling the sequential FPL team selection process as a Belief-State Markov Decision Process
- For the $i$-th gameweek, we define the following terms:
    - $M_i$ is set of matches in gameweek $i$.
    - $P_i$ is the set of players available for selection in gameweek $i$.
    - $A_i$ is the set of actions available in gameweek $i$, where $a \in A_i$ is a subset of $P_i$ and observes all team selection constraints.
    - $p_i \in P_i$ is associated with its FPL-designated position $pos(p_i)$ and price $pr(p_i)$.
    - $\tau_p \in \tau$ is a system of distributions representing player's performance/influence on the matchplay.
    - $O_i$ is the set of match observations in gameweek $i$
    - $o \in O_i$ includes both the result of the matches and the performance of the players in the selected team e.g. goals, assists, clean sheets, yellow cards, red cards, bonus points. The probability of each $o \in O_i$ is somehow dependent on the players' characteristics ($\tau$) i.e. a team with strong attackers is more likely to score goals, therefore, $P(o | \tau)$ is dependent on $\tau$.
    - $R(o, a_{prev}, a_{curr})$ is the reward function, which returns the points scored by the selected team $a_{curr}$, given the match observations $o$. The previous team $a_{prev}$ is also provided to penalize the agent for any player poor player transfers or transfers beyond the allowed number.

Markov's Decision Process (MDP) 
- A state $S_i$ would encapsulate
    - $M_{i,..., 38}$ - set of upcoming fixtures for that gameweek
    - $P_i$ - set of players available for selection
    - $o \in O_{i - 1}$ - the outcome of the previous gameweek
    - $\tau$ - the system of distributions representing players' abilities
- An action $A_i$ is the set of teams selectable in gameweek $i$
- $R$ is the corresponding reward function

Belief model ($\tau$):
- Represent uncertainty over players' abilities and generate samples $\tau$ from the distribution $Pr(\tau | b)$.
- Three distributions are used to model the players' abilities:
    - $\rho_p$ - a three-state categorical distribution representing the player's probability of starting a match, being substituted, or not playing at all i.e. (start, sub, unused).
    - $\omega_p$ - a Bernoulli/Binomial distribution over a single trial, representing the probability of a player scoring a goal given he was playiong at the time
    - $\psi_p$ - a Bernoulli distribution representing the probability of a player providing an assist given he was playing at the time
- Define prior distributions over the parameters of the above distributions and update them using the match observations $o$ to obtain new posterior distributions.
- Use simple closed-form equations e.g. Beta and Dirichlet conjugate priors to update the priors.
- Sample from these conjugate distributions to generate $\tau_p$.
- Define hyperparemeters uniformly across all players i.e. $$\omega_p \sim Beta(1, 1), \psi_p \sim Beta(1, 1),  \rho_p \sim Dirichlet(\frac{1}{4}, \frac{1}{4}, \frac{1}{4})$$
- Potential to use performance data from previous seasons to define priors
- Define 4 global multinomial distributions $S_{pos}$ - one for each position - to describe the distribution of minutes players who play the same position $pos$ are likely to play in a match, given they start the match.
- Player absence via injury/suspension or any other reson is modelled by setting the probability of starting and substituting to zero i.e. $Pr(\rho_p = start) \text{and} Pr(\rho_p = sub) = 0$.

#### Formulating the belief-state MDP
- The belief state at gameweek $i$, $b_i$, is an instantiation of our belief model, updated with the match observations $O_{i - 1}$.
- We observe the posterior player characteristics by updating the belief state in response to an observation $o \in O_i$ via the Bayes rule: $$Pr(\tau | b_{i + 1}) \propto Pr(o | \tau)Pr(\tau | b_i)$$
- The agent can perform optimally by maximizing the value of the Bellman equation: $$V(b_i) = \max_{a \in A_i} Q(b_i, a)$$
- The Q-function is defined as: $$Q(b_i, a) = \int_{\tau} Pr(\tau | b_i)  \int_{o \in O_i} Pr(o | \tau) \left[r_i + \gamma V(b_{i + 1}) \right] \text{dod}\tau$$
- Where:
    - $\gamma \in [0, 1)$ is the discount factor for future rewards
    - $r_i = R(o, a_{prev}, a)$ is the reward function
    - $V(b_{i + 1})$ is the value of the next belief state
- Solutions to the Bellman equation is intractable due to the size of the outcome space $|O_i|$, the size of the action space $|A_i|$, and the need to consider up to 38 gameweeks in order to calculate Q-values exactly.
- We can work around this sampling from $O_i$ and simulating match outcomes to approximate the Q-function


#### Sampling Outcomes
- We describe a model for sampling outcomes for gameweek $i$ from $Pr(O_i | \tau)$. This is then combined with the belief model described beforehand to obtain a joint distribution of player abilities and match outcomes, thus treating uncertainty in player abilities in a Bayesian manner (observations) $$Pr(O_i | \tau)Pr(\tau | b_i)$$
- Sampling procedure for a single match that also extends to any other match in the gameweek (it also takes the perspective of the home team, which naturally extends to the away team as well):
    - Define $P_H$ and $P_A$ as the set of players available the home and away teams respectively.
    - Sample $\tau_p$ for each player $p \in P_H$  from the belief model $Pr(\tau_p | b_i)$
    - Randomly select eleven players from $P_H$ in proportion to their probability of starting the match i.e. $Pr(\rho_p = start)$
        - These players constitute the starting lineup $L_H$
    - The minute each player leaves the pitch is sampled from the $S_{pos}$ distribution for the player's position
    - Each player in $P_H$ and not in $L_H$ is assigned to the set of substitutes $U_H$
        - At the start of each minute of the match, we check if any player in $L_H$ is scheduled to be substituted
        - If so, we randomly select a player from $U_H$ to replace the outgoing player in proportion to the probability of the player being substituted i.e. $Pr(\rho_p = sub)$
        - The replacement is added to $L_H$ (removed from $U_H$). We further assume that the player being substituted is not substituted again in the same match.
        - If a goal is scored according to the underlying team-based model, then it is allocated to player $p$ with probability $Pr(\omega_p = 1)$ while an assist is allocated to player $p$ with probability $Pr(\psi_p = 1)$.
    - These point estimates may then be used in combination with the MDP reward function $R$ to approximate the immediate reward from performing any action, as well as to guide the exploration of high quality regions of the action space.


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from constants import GAMEWEEK_COUNT, DATA_FOLDER
from gameweek_simulator import GameweekSimulator
from player_ability import PlayerAbility
from position_minutes import PositionMinutesModel
from fpl_env import FPLEnv
from bayesianFPLAgent import BayesianFPLAgent
from pprint import pprint
from utils import update_gw_data
from data_registry import DataRegistry


In [13]:
SEASON_START_YEAR = "2023"

fixtures_2023_24 = pd.read_csv(filepath_or_buffer=f"{DATA_FOLDER}/2023-24/fixtures.csv")
gameweek_data = DataRegistry(gw_data_columns=["name", "total_points", "value", "GW"]).gameweek_data["2023-24"]


# Bayesian model for players' ability to (start, get subbed, or not play), score, and assist
player_ability_model = PlayerAbility(season_start_year=SEASON_START_YEAR, gameweek="0")

# Bayesian model for players' game minutes per position 
position_minutes_model = PositionMinutesModel()

cumulative_real_player_points = {
    player_name: 0
    for player_name in player_ability_model.player_ability.name
}

for gw_count in range(1, GAMEWEEK_COUNT + 1, 1):
    # Filter fixtures and players available for specified gameweek
    fixtures_2023_24_gameweek = fixtures_2023_24[fixtures_2023_24["GW"] == gw_count]
    # Small caveat for GW since no previous gameweek to reference for prices only; not points (will be 0)
    gameweek_data_current_gw = gameweek_data[gameweek_data["GW"] == gw_count] if (
        gw_count == 1) else gameweek_data[gameweek_data["GW"] == (gw_count - 1)]

    # player_points_df = GameweekSimulator().simulate_gameweek(
    #     season_start_year=SEASON_START_YEAR, 
    #     gameweek=str(gw_count),
    #     fixtures_df=fixtures_2023_24_gameweek,
    #     players_ability_df=player_ability_model.player_ability[
    #         player_ability_model.player_ability.name.isin(gameweek_data_current_gw.name)
    #     ],
    #     position_minutes_df=position_minutes_model.model_df
    # )
    # player_points_df.to_csv("../data/2023-24/model_data/player_points_df.csv")
    player_points_df = pd.read_csv("../data/2023-24/model_data/player_points_df.csv")
    # Update players' cumulative real points
    player_points_df, cumulative_real_player_points = update_gw_data(
        gameweek_data=gameweek_data_current_gw,
        player_points_df=player_points_df, 
        cumulative_real_player_points=cumulative_real_player_points,
        gw=gw_count,
    )
    reinforcement_learning_env = FPLEnv(
        player_performance_samples=player_points_df,
        initial_budget=100.0,
        max_transfers_per_gw=1,
        transfer_penalty=4
    )
    agent = BayesianFPLAgent()
    results, state_info = agent.train(env=reinforcement_learning_env, num_episodes=100)
    pprint(state_info)
    print(f"Gameweek: {gw_count}; Projected points: {results[-1]}")
    print("---------------------------------------------------------------------------------------")
    # Update player ability priors with completed gameweek's data
    player_ability_model.update_model()
    break

Player abilities matched players: 585


  0%|          | 0/100 [00:00<?, ?it/s]


15


KeyError: "['Aaron Connolly'] not found in axis"